<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Semantic_Features_3DVision_2020_City_Scapes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Our mapping pipeline is of the following structure:

1.   Detection of objects of certain object classes (e.g. traffic sign). Output: object bounding boxes
2.   Triangulation of objects. Output: object position relative to pose
3.   Creating map of objects. (And refining with filter, BA, etc.) Output: List of objects and their corresponding positions
4.   Visualizing map.





We start by loading the benchmark dataset. The Cityscapes dataset is used. 

Scripts for analyzing the dataset can be found here: https://github.com/mcordts/cityscapesScripts

How to download the zip files directly: https://towardsdatascience.com/download-city-scapes-dataset-with-script-3061f87b20d7



Download CityScape files: 

*   leftImg8bit_trainextra.zip
*   disparity_trainextra.zip
*   camera_trainextra.zip
*   vehicle_trainextra.zip

The files are unzipped into data/... respectively.









In [1]:
# Download and unzip (run only once!).
PATH_TO_LEFT_IMAGES = "leftImg8bit/train_extra/schweinfurt/"
PATH_TO_DISPARITY = "disparity/train_extra/schweinfurt/"
PATH_TO_CAMERA = "camera/train_extra/schweinfurt/"
PATH_TO_VEHICLE = "vehicle/train_extra/schweinfurt/"

# Remove data directory if it is already loaded (if needed).
#!rm -r data
# Login.
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=ftaubner@ethz.ch&password=semantic_dudes&submit=Login' https://www.cityscapes-dataset.com/login/
# Get left camera images. I put it on OneDrive so that only Schweinfurt has to be downloaded.
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199734&authkey=AC5K24PFcrSPFl4" -O leftImg8bit_trainextra.zip
# Alternatively, it can be downloaded from CityScapes website directly:
#   (!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=4)
# Extract.
!mkdir data
!unzip -qq leftImg8bit_trainextra.zip "$PATH_TO_LEFT_IMAGES"* -d data
# And delete.
!rm leftImg8bit_trainextra.zip

# Get disparity maps.
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199735&authkey=AEwGRlH_TySnyPM" -O disparity_trainextra.zip
# Original file:
#   (!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=22)
!unzip -qq disparity_trainextra.zip "$PATH_TO_DISPARITY"* -d data
!rm disparity_trainextra.zip
# Get camera intrinsics. 
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=9
!unzip -qq camera_trainextra.zip "$PATH_TO_CAMERA"* -d data
!rm camera_trainextra.zip
# Get vehicle odometry.
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=11
!unzip -qq vehicle_trainextra.zip "$PATH_TO_VEHICLE"* -d data
!rm vehicle_trainextra.zip

import os
PATH_TO_LEFT_IMAGES = os.path.join("data/", PATH_TO_LEFT_IMAGES)
PATH_TO_DISPARITY = os.path.join("data/", PATH_TO_DISPARITY)
PATH_TO_CAMERA = os.path.join("data/", PATH_TO_CAMERA)
PATH_TO_VEHICLE = os.path.join("data/", PATH_TO_VEHICLE)

print("Finished unzipping.")

--2020-04-17 10:43:52--  https://www.cityscapes-dataset.com/login/
Resolving www.cityscapes-dataset.com (www.cityscapes-dataset.com)... 139.19.217.8
Connecting to www.cityscapes-dataset.com (www.cityscapes-dataset.com)|139.19.217.8|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.cityscapes-dataset.com/downloads/ [following]
--2020-04-17 10:43:53--  https://www.cityscapes-dataset.com/downloads/
Reusing existing connection to www.cityscapes-dataset.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html’

index.html              [ <=>                ]  42.77K  --.-KB/s    in 0.1s    

2020-04-17 10:43:55 (393 KB/s) - ‘index.html’ saved [43792]

--2020-04-17 10:43:58--  https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199734&authkey=AC5K24PFcrSPFl4
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)

We start with the detection of objects of interest. Faster R-CNN is used for this task. (Julius)

In [0]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
static_classes = []              

Loading an on COCO pre-trained Faster R-CNN 


Installing Seamless Scene Segmentation from https://github.com/mapillary/seamseg
For Seamless Scene Segmentation a pre-trained version on the Mapillary Dataset exists. Dataloaders for Cityscapes seem to exist, too.

It is a network for panoptic segmentation basen on Mask-R-CNN. Alternatively the panoptic variant in the detectron2 reopisitory can also be trained on Cityscapes or Mapillary. Dataloaders seem to exist for both Datasets. 

In [0]:
#@title
#!pip install git+https://github.com/mapillary/seamseg.git
#!pip install wget

#url = 'https://drive.google.com/file/d/1ULhd_CZ24L8FnI9lZ2H6Xuf03n6NA_-Y/view'
#!wget $url


These functions convert a difference in coordinates (lat., long.) to a difference in metric frame (x, y) and vice versa. Both conversions are dependent on the current latitude.
Under the assumption that we are in Europe, east of 0°!

Approximative conversions:

* Latitude: 1 deg = 110.574 km
* Longitude: 1 deg = 111.320*cos(latitude) km


In [0]:
# Felix
# Helper functions for transformations.

import numpy as np


# Calculates the difference between two coordinates in meters (x to east, y to north).
# See below for example.
def coord_diff_to_metric_diff(d_lat, d_long, lat):
  d_y = d_lat * 110574
  d_x = d_long * 111320 * np.cos(np.radians(lat))
  return d_x, d_y


# Converts the metric difference to differences in latitude and longitude.
# See below for example.
def metric_diff_to_coord_diff(d_x, d_y, lat):
  d_lat = d_y / 110574
  d_long = d_x / 111320 / np.cos(np.radians(lat))
  return d_lat, d_long


# Returns the transformation matrix: world frame (-> vehicle frame) -> camera frame
# Usage: [x_world; 1] = camera_frame_to_world_transform(args).dot([x_camera_frame; 1])
# All angles in radians, distances in meters!
# See below for example.
def camera_frame_to_world_transform(heading, yaw_ext, pitch_ext, roll_ext, x_ext, y_ext, z_ext):
    # Coordinate axis switch for camera.
    C_c = np.array([[0., -1.,  0., 0.],
                    [0.,  0., -1., 0.],
                    [1.,  0.,  0., 0.],
                    [0.,  0.,  0., 1.]])
    # Vehicle to camera transformation matrix.
    c_y = np.cos(yaw_ext)
    c_p = np.cos(pitch_ext)
    c_r = np.cos(roll_ext)
    s_y = np.sin(yaw_ext)
    s_p = np.sin(pitch_ext)
    s_r = np.sin(roll_ext)
    T_v_c = np.array([[c_y*c_p, c_y*s_p*s_r-s_y*c_r, c_y*s_p*c_r+s_y*s_r, x_ext],
                      [s_y*c_p, s_y*s_p*s_r+c_y*c_r, s_y*s_p*c_r-c_y*s_r, y_ext],
                      [   -s_p,             c_p*s_r,             c_p*c_r, z_ext],
                      [     0.,                  0.,                  0.,    1.]])

    # World to vehicle transformation matrix.
    c_h = np.cos(heading)
    s_h = np.sin(heading)
    T_w_v = np.array([[c_h, -s_h, 0., 0.],
                      [s_h,  c_h, 0., 0.],
                      [ 0.,   0., 1., 0.],
                      [ 0.,   0., 0., 1.]])

    # Return camera space to vehicle transform.
    return T_w_v.dot(T_v_c.dot(np.linalg.inv(C_c)))

In [4]:
# Testing and example (taken from Johannes' map pic on WhatsApp):
# Converting coords into metrics:
d_x, d_y = coord_diff_to_metric_diff(48.370-48.369, 10.896-10.894, 48.368)
print("Delta x in meters: {}".format(d_x))
print("Delta y in meters: {}".format(d_y))

# Converting back to coords: should be the same:
d_lat, d_long = metric_diff_to_coord_diff(d_x, d_y, 48.368)
print("Difference in latitude: {}".format(d_lat))
print("Difference in longitude: {}".format(d_long))

# Convert a camera frame coordinate to world coordinate:
x_example = np.array([[8.1], [0.7], [0.7]])
x_world_example = camera_frame_to_world_transform(0.25, 0.0057, 0.055, 0.0, 1.7, -0.1, 1.3).dot(np.vstack((x_example, 1.0)))
print("Camera coordinate: ")
print(x_example)
print("World coordinate: ")
print(x_world_example)

Delta x in meters: 147.90949435332325
Delta y in meters: 110.5739999997423
Difference in latitude: 0.0009999999999976694
Difference in longitude: 0.002000000000000668
Camera coordinate: 
[[8.1]
 [0.7]
 [0.7]]
World coordinate: 
[[ 4.35955249]
 [-7.34589949]
 [ 0.56257789]
 [ 1.        ]]


In [0]:
# Mapping Algorithm Helper Functions

import math

# Convert degree to radian
def degree_to_rad_converter(degree_value):
  rad_value = degree_value * math.pi / 180
  return rad_value

# Calculate real 3D position
def pixel_to_3Dposition_converter(x, depth, focal):
  x_3D = x * depth / focal
  return x_3D

# Calculate box center to get average disparity
def boundaries_to_box_center(x_1, x_2):
  x_center = (x_1 + x_2)/2
  return x_center

# Get camera focal length and baseline (To be corrected, pixels are NOT quadratic here fx =/= fy)
def camera_parameters(fx, fy):
  # Take average, which is Bullshit, but not better idea yet
  # focal = (fx + fy)/2
  focal = fx
  # Get baseline value from calibration data
  baseline = 0.222384
  return focal, baseline

# Get disparities of all points (pixels)
def disparity_catcher(x, y, img):
  # DATA_DIR="data/disparity/train_extra/schweinfurt"
  # im_directory = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
  # n_im = len(im_directory)


  disp = img[y, x]
  if disp != 0 and disp > 5125:
    disp = (disp - 1.) / 256.

    # print(disp)
    return disp
  else:
    return "nan"


# Convert disparity to actual depth information
def disparity_to_depth(disparity_box_center):
  if disparity_box_center > 0:
    depth_center = baseline * focal / disparity_box_center
  else:
    depth_center = 100
  return depth_center

# Transformation to world frame coordinates
# _3dImage	=	cv.reprojectImageTo3D('data/disparity/train_extra/schweinfurt/schweinfurt_000000_000000_disparity.png', Q)


In [0]:
# Get GPS Data from json files
import pandas as pd 
import os
import json
import glob
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR="data/vehicle/train_extra/schweinfurt"
vehicle_datas_path = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
n_images = len(vehicle_datas_path)
vehicle_data_path_list = []
images_path = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
n_images = len(images_path)

r = 0
i = 0

for r in range(n_images):
    vehicle_data_path_list.append('data/vehicle/train_extra/schweinfurt/schweinfurt_000000_000'+ str(f"{r:0>3}") + '_vehicle.json')

# create space for long at lat
gpsLatitude_actual = np.zeros(n_images)
gpsLongitude_actual = np.zeros(n_images)
gpsHeading_actual = np.zeros(n_images)
gpsLatitude_delta = np.zeros(n_images)
gpsLongitude_delta = np.zeros(n_images)
gpsHeading_delta = np.zeros(n_images)
gpsHeading_actual_rad = np.zeros(n_images)


for vehicle_data in vehicle_data_path_list:
  # Load json from path
  with open(vehicle_data) as json_file:
    data = json.load(json_file)
    gpsLatitude_actual[i] = data['gpsLatitude']
    gpsLongitude_actual[i] = data['gpsLongitude']
    gpsHeading_actual[i] = data['gpsHeading']
    gpsHeading_actual_rad[i] = degree_to_rad_converter(gpsHeading_actual[i])
    if i > 0:
      gpsLatitude_delta[i] = gpsLatitude_actual[i] - gpsLatitude_actual[i-1]
      gpsLongitude_delta[i] = gpsLongitude_actual[i] - gpsLongitude_actual[i-1]
      gpsHeading_delta[i] = gpsHeading_actual[i] - gpsHeading_actual[i-1]
  i+=1

# Actual coordinates in x, y starting at 0, 0

gps_step_delta_X = np.zeros(n_images)
gps_total_delta_X = np.zeros(n_images)
gps_step_delta_Y = np.zeros(n_images)
gps_total_delta_Y = np.zeros(n_images)
gps_total_delta_Heading = np.zeros(n_images)

i = 0

for image_path in images_path:
  if i > 0:
    gps_step_delta_X[i], gps_step_delta_Y[i] = coord_diff_to_metric_diff(gpsLatitude_delta[i], gpsLongitude_delta[i], gpsLatitude_actual[i])
    gps_total_delta_X[i] = gps_total_delta_X[i-1] + gps_step_delta_X[i]
    gps_total_delta_Y[i] = gps_total_delta_Y[i-1] + gps_step_delta_Y[i]
    gps_total_delta_Heading[i] = gps_total_delta_Heading[i-1] + gpsHeading_delta[i]
  i+=1

i = 0


In [0]:
# Get Camera Calibration Data

DATA_DIR="data/camera/train_extra/schweinfurt"
cam_datas_path = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
n_images = len(cam_datas_path)
cam_data_path_list = []

r = 0
i = 0

for i in range(n_images):
    cam_data_path_list.append('data/camera/train_extra/schweinfurt/schweinfurt_000000_000'+ str(f"{i:0>3}") + '_camera.json')

# create space for long at lat
roll  = np.zeros(n_images)
pitch = np.zeros(n_images)
yaw = np.zeros(n_images)
baseline = np.zeros(n_images)
offset_x = np.zeros(n_images)
offset_y = np.zeros(n_images)
offset_z = np.zeros(n_images)
f_x = np.zeros(n_images)
f_y = np.zeros(n_images)
u_0 = np.zeros(n_images)
v_0 = np.zeros(n_images)

for cam_data_path in cam_data_path_list:
  # Load json from path
  with open(cam_data_path) as json_file:
    data = json.load(json_file)
    roll[r] = data['extrinsic']['roll']
    pitch[r] = data['extrinsic']['pitch']
    yaw[r] = data['extrinsic']['yaw']
    baseline[r] = data['extrinsic']['baseline']
    offset_x[r] = data['extrinsic']['x']
    offset_y[r] = data['extrinsic']['y']
    offset_z[r] = data['extrinsic']['z']
    f_x[r] = data['intrinsic']['fx']
    f_y[r] = data['intrinsic']['fy']
    u_0[r] = data['intrinsic']['u0']
    v_0[r] = data['intrinsic']['v0']
  r+=1


In [0]:
# Get Results File from Julius (Run only once !)

import shutil
import os

!wget 'https://raw.githubusercontent.com/juliuserbach/Semantic-Features/master/results.json' -O results.json


path = "data_KITTI"
os.mkdir(path)
path = "data_KITTI/Numerical_Results"
os.mkdir(path)
shutil.copy("results.json", "data_KITTI/Numerical_Results")

# Download and unzip (run only once!).
PATH_TO_CALIB = "dataset/sequences/"
PATH_TO_POSE = "dataset/poses/"

!wget "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21100433&authkey=APrMUGQyaB4np4Q" -O kitti_dataset.zip

!mkdir data_KITTI
!unzip -qq kitti_dataset.zip "$PATH_TO_CALIB"* -d data_KITTI
!unzip -qq kitti_dataset.zip "$PATH_TO_POSE"* -d data_KITTI

# And delete.
!rm kitti_dataset.zip


In [10]:
# Mapping Algorithm -> Associates objects with respective coordinates in world coordinate frame and stores them in vector containing Long, Lat,
#                      Height, class and corresponding image ID

import pandas as pd
import math
import csv
import pandas as pd
import cv2
import glob

# Ask which disparity version to take from disparity matrix. Options are: 0 for median or 1 for maximum

option = input("Enter your option (0 = median, 1 = maximum): ")
option = int(option)

k = 0 # Images
t = 0 # Total valid object number

# Get focal length from calibration data
fx = 2273.82
fy = 2236.9989965173254
# baseline = 0.222384

focal, baseline = camera_parameters(fx, fy)


x_3D_rel_car = []
y_3D_rel_car = []
z_3D_rel_car = []
gpslat = []
gpslong = []
relz = []
semanticlabel = []
imageid = []

results_directory = '/content/data_KITTI/Numerical_Results/results.json'

# Load json from path
with open(results_directory) as json_file:
  data = json.load(json_file)
  for i in range(len(data['results'])):
    print("Processing image number", k)
    print(x_3D_rel_car)
    n_objects = int(len(data['results'][k]['classes']))
    # Get image number
    image_no = data['results'][k]['image_id'][3:6]
    print(image_no)
    im_no = int(image_no)
    # disparity_data_path = 'data/disparity/train_extra/schweinfurt/schweinfurt_000000_000'+ image_no + '_disparity.png'
    disparity_data_path = 'data_KITTI/dataset/sequences/04/disparity_2/' + image_no + 'raw.png'

    # Load image from path
    img = cv2.imread(disparity_data_path, cv2.IMREAD_UNCHANGED)

    for o in range(n_objects):
      # Get coordinates in image frame
      u1 = data['results'][k]['boxes'][o][0]
      v1 = data['results'][k]['boxes'][o][1]
      u2 = data['results'][k]['boxes'][o][2]
      v2 = data['results'][k]['boxes'][o][3]

      # Calculate center point in image frame
      u_cent = int(boundaries_to_box_center(u1, u2))
      v_cent = int(boundaries_to_box_center(v1, v2))

      # Get and write all disparities 1/10th of box-length around center point in matrix
      disp_matrix = np.zeros(shape=(int(abs((u_cent-u1)/1)),int(abs((v_cent-v1)/1))))
        
      for y_coord in range(int(abs((v_cent-v1)/1))):
        for x_coord in range(int(abs((u_cent-u1)/1))):
          #print(disparity_catcher(x_coord + int(u_cent) - int(abs(u_cent - u1)/10), y_coord + int(v_cent) - int(abs(v_cent - v1)/10), img))
          disp_matrix[x_coord][y_coord] = disparity_catcher(x_coord + int(u_cent) - int(abs(u_cent - u1)/2), y_coord + int(v_cent) - int(abs(v_cent - v1)/2), img)
        
      # Take median of disparity values
      
      if option == 0:
        med_disp = np.nanmedian(disp_matrix, axis=None, out=None, overwrite_input=False, keepdims=False)
      if option == 1:
        try:
          med_disp = np.nanmax(disp_matrix, axis=None, out=None, keepdims=False)
        except RuntimeWarning:
          med_disp = np.nanmax(disp_matrix, axis=None, out=None, keepdims=False)
        #else:
          #med_disp = np.nanmax(disp_matrix, axis=None, out=None, keepdims=False)

      if med_disp != 0:

        # Get disparity of center point in image frame
        # disp_cent = disparity_catcher(u_cent, v_cent, k)

        # Calculate depth from disparity
        depth_med = disparity_to_depth(med_disp)

        # Transform coordinates from image plane to camera frame
        x1 = depth_med * (u1 - u_0[im_no]) / f_x[im_no]
        y1 = depth_med * (v1 - v_0[im_no]) / f_y[im_no]
        x2 = depth_med * (u2 - u_0[im_no]) / f_x[im_no]
        y2 = depth_med * (v2 - v_0[im_no]) / f_y[im_no]

        # Calculate box center in camera frame
        x_cent = boundaries_to_box_center(x1, x2)
        y_cent = boundaries_to_box_center(y1, y2)

        # Save object coordinates in camera frame
        x_3D_rel_car.append(x_cent)
        y_3D_rel_car.append(y_cent)
        z_3D_rel_car.append(depth_med)

        # Transform object coordinates to world frame
        input_vector = np.array([[x_3D_rel_car[t]], [y_3D_rel_car[t]], [z_3D_rel_car[t]]])
        output_vector = camera_frame_to_world_transform(gpsHeading_actual_rad[im_no], yaw[im_no], pitch[im_no], roll[im_no], offset_x[im_no], offset_y[im_no], offset_z[im_no]).dot(np.vstack((input_vector, 1.0)))
        
        # Transform back to Lat / Long and save to vector
        new_x = -output_vector[1]
        new_y = output_vector[0]
        d_lat, d_long = metric_diff_to_coord_diff(new_x, new_y, gpsLatitude_actual[im_no])
        
        #dataframe
        gpslat.append(gpsLatitude_actual[im_no] + d_lat)
        gpslong.append(gpsLongitude_actual[im_no] + d_long)
        relz.append(output_vector[2])
        semanticlabel.append(data['results'][k]['classes'][o])
        imageid.append(data['results'][k]['image_id'][3:6])
      
        t += 1
    k += 1

  mapping_output_df = pd.DataFrame({'imageID': imageid, 'longitude': gpslong, 'latitude': gpslat, 'z': relz, 'semanticlabel': semanticlabel})

  for i in mapping_output_df:
    mapping_output_df[i] = np.squeeze(mapping_output_df[i].tolist())


Enter your option (0 = median, 1 = maximum): 1
Processing image number 0
[]
163


IndexError: ignored

In [0]:
### Writes Data to DataFrame and to csv file

# Create DataFrame 


# Print the output
print(mapping_output_df)

# Write output to csv file
mapping_output_df.to_csv(r'data/mapping_output.csv', index = False)

In [14]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

!pip install git+https://github.com/utiasSTARS/pykitti.git

import pykitti

# Change this to the directory where you store KITTI data
basedir = '/content/data_KITTI/dataset'

# Specify the dataset to load
sequence = '04'

# Load the data. Optionally, specify the frame range to load.
# dataset = pykitti.odometry(basedir, sequence)
dataset = pykitti.odometry(basedir, sequence, frames=range(0, 20, 5))

# dataset.calib:      Calibration data are accessible as a named tuple
# dataset.timestamps: Timestamps are parsed into a list of timedelta objects
# dataset.poses:      List of ground truth poses T_w_cam0
# dataset.camN:       Generator to load individual images from camera N
# dataset.gray:       Generator to load monochrome stereo pairs (cam0, cam1)
# dataset.rgb:        Generator to load RGB stereo pairs (cam2, cam3)
# dataset.velo:       Generator to load velodyne scans as [x,y,z,reflectance]

# Grab some data
first_pose = dataset.poses[0]
second_pose = dataset.poses[1]

print(first_pose)
print(second_pose)

# To do:
# - Get Position of car
# - Get Camera Calibration data of left camera
# - Do normal camera calibration but with new data
# - Get disparity using new file
# - Add new position data onto coordinates


# Load the data. Optionally, specify the frame range to load.
# dataset = pykitti.odometry(basedir, sequence)
dataset = pykitti.odometry(basedir, sequence)

# Grab some data
second_pose = dataset.poses[219]
first_rgb = dataset.get_rgb(219)
first_cam2 = dataset.get_cam2(219)
velo = dataset.get_velo(219)
baseline = dataset.calib.b_rgb

disparity = cv2.imread("/home/felix/vision_ws/Semantic-Features/content/kitti_dataset/dataset/sequences/04/rawpng/219raw.png",
                           cv2.IMREAD_UNCHANGED)

P_cam2 = dataset.calib.P_rect_20
T_cam2_velo = dataset.calib.T_cam2_velo


  Cloning https://github.com/utiasSTARS/pykitti.git to /tmp/pip-req-build-f67i6qyn
  Running command git clone -q https://github.com/utiasSTARS/pykitti.git /tmp/pip-req-build-f67i6qyn
  Created wheel for pykitti: filename=pykitti-0.3.1-cp36-none-any.whl size=13234 sha256=a90fc85640e094639f8e94592b84a88c12dce5bfd672858c753baff6fd17e1f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-qe93mglb/wheels/4b/55/6f/2986578a5cf7afc0ea7e14cfff9cf5bc8e625e9c4e553f02f9
Successfully built pykitti
[[ 1.000000e+00  1.197625e-11  1.704638e-10 -5.551115e-17]
 [ 1.197625e-11  1.000000e+00  3.562503e-10  0.000000e+00]
 [ 1.704638e-10  3.562503e-10  1.000000e+00  2.220446e-16]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
[[ 9.999922e-01 -3.925219e-03 -5.162240e-04  4.157928e-03]
 [ 3.926062e-03  9.999909e-01  1.640787e-03 -1.014678e-01]
 [ 5.097792e-04 -1.642800e-03  9.999985e-01  6.581638e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
